<a href="https://colab.research.google.com/github/K-RLange/relatio/blob/master/Relatio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd /content/gdrive/My Drive/Relatio
import pandas as pd
from tqdm import tqdm
import pickle

Mounted at /content/gdrive
/content/gdrive/My Drive/Relatio


In [7]:
! python -m pip install -U pip wheel setuptools

# install the package
!pip install -U git+https://github.com/K-RLange/relatio.git
#! python -m pip install -U relatio
! python -m pip install -U pandas
! python -m pip install --upgrade google-cloud-storage

# download SpaCy and NLTK additional resources
! python -m spacy download en_core_web_sm
! python -m nltk.downloader punkt wordnet stopwords averaged_perceptron_tagger
from relatio.utils import split_into_sentences
from relatio.wrappers import run_srl
import nltk

nltk.download("stopwords")
nltk.download("punkt")
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from relatio.wrappers import get_narratives
import spacy
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
from relatio.wrappers import build_narrative_model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 8.7 MB/s 
     |████████████████████████████████| 1.2 MB 59.8 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/K-RLange/relatio.git to /tmp/pip-req-build-0t9n9c1g
  Running command git clo

ImportError: ignored

In [ ]:
data = pickle.load(open("coref_texts.pickle", "rb"))#pd.read_csv("InflationParagraphs6.csv", encoding="unicode_escape").x
sub_data = data#[x[0] for x in data[0:50]]# if "inflation" in x or "Inflation" in x[0]]
df = pd.DataFrame({"id": [x for x in range(len(sub_data))], "doc": [x for x in sub_data]})

In [ ]:
split_sentences = split_into_sentences(
    df, progress_bar=True
)

In [ ]:
split_sentences = pickle.load(open("sentences.pickle", "rb"))
srl_res = run_srl(
    path="https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz",  # pre-trained model
    sentences=split_sentences[1],
    progress_bar=True,
)

downloading: 100%|##########| 51.7M/51.7M [00:02<00:00, 20.7MiB/s]


Running SRL...


100%|██████████| 1554/1554 [4:09:43<00:00,  9.64s/it]


In [ ]:
#list(map(len, srl_res))
pickle.dump(split_sentences, open("sentences.pickle", "wb"))
pickle.dump(srl_res, open("srl.pickle", "wb"))

In [3]:
split_sentences = pickle.load(open("sentences.pickle", "rb"))
srl_res = pickle.load(open("srl.pickle", "rb"))

In [ ]:
srl_res[1429]

In [5]:
narrative_model = build_narrative_model(
    srl_res=srl_res,
    sentences=split_sentences[1],
    embeddings_type="gensim_keyed_vectors",  # see documentation for a list of supported types
    embeddings_path="glove-wiki-gigaword-300",
    n_clusters=[[2]],
    top_n_entities=100,
    stop_words=spacy_stopwords,
    remove_n_letter_words=1,
    progress_bar=True,
)
print(narrative_model.keys())
narrative_model['entities'].most_common()[:20]

Processing SRL...


100%|██████████| 216520/216520 [00:42<00:00, 5104.65it/s]


finished
Cleaning SRL...


100%|██████████| 905554/905554 [14:25<00:00, 1046.36it/s]


Computing role frequencies...


...

100%|██████████| 216520/216520 [25:11<00:00, 143.24it/s]


Mapping named entities...


100%|██████████| 905554/905554 [02:00<00:00, 7497.35it/s]


Loading embeddings model...
[==================================================] 100.0% 376.1/376.1MB downloaded
dict_keys(['roles_considered', 'roles_with_entities', 'roles_with_embeddings', 'dimension_reduce_verbs', 'clean_text_options', 'verb_counts', 'entities', 'top_n_entities', 'embeddings_model', 'cluster_model', 'cluster_labels_most_similar', 'cluster_labels_most_freq'])


[('fed', 23587),
 ('china', 5733),
 ('american', 5449),
 ('united state', 5410),
 ('european', 2806),
 ('republican', 2297),
 ('federal reserve', 2227),
 ('chinese', 1974),
 ('na', 1851),
 ('japan', 1838),
 ('powell', 1827),
 ('treasury', 1798),
 ('congress', 1711),
 ('germany', 1572),
 ('bernanke', 1533),
 ('britain', 1400),
 ('trump', 1352),
 ('democrat', 1253),
 ('german', 1189),
 ('new york', 1112)]

In [6]:
pickle.dump(narrative_model, open("model.pickle", "wb"))

In [7]:
final_statements = get_narratives(
    srl_res=srl_res,
    doc_index=split_sentences[0],  # doc names
    narrative_model=narrative_model,
    n_clusters=[0],
    progress_bar=True,
)

Processing SRL...


100%|██████████| 216520/216520 [00:37<00:00, 5723.15it/s]


finished
Cleaning SRL...


100%|██████████| 905554/905554 [14:22<00:00, 1050.22it/s]


Processing raw arguments...


...

Mapping named entities...


100%|██████████| 905554/905554 [02:04<00:00, 7289.95it/s]


Assigning clusters to roles...


100%|██████████| 905554/905554 [04:28<00:00, 3370.80it/s]


test


In [8]:
#list(map(len, srl_res))
pickle.dump(narrative_model, open("model.pickle", "wb"))
pickle.dump(final_statements, open("final_statements.pickle", "wb"))

In [3]:
#narrative_model = pickle.load(open("model.pickle", "rb"))
final_statements = pickle.load(open("final_statements.pickle", "rb"))

In [ ]:
final_statements[final_statements["doc"] == 0]

In [36]:
print(final_statements.head())
def extend_inside(lists):
    new_list = []
    for element in lists:
        for part_of_element in element:
            new_list.append(part_of_element)
    return new_list

#Detect maximum number of statements within a sentence using a greedy-approach
#of finding the maximum number of disjoint statements. Starts with the statement
#that ends first of all statements
def max_disjoint_intervals(statements, doc, sentence, condition=["doc"]):
    indices = []
    list_ = [[statements.loc[x]["statement"], statements.loc[x]["start"], statements.loc[x]["end"]] for 
             x in statements.index if statements.loc[x]["sentence"] == sentence
             and all([statements.loc[x][y] for y in condition])]
    if len(list_) > 0:
        # Lambda function to sort the list
        list_.sort(key = lambda x: x[2])
        indices.append(list_[0][0])

        # End point of first interval
        r1 = list_[0][2]
        
        used = False
        for i in list_:
            used = False
            l1 = i[1]
            r2 = i[2]

            if l1 > r1:
                indices.append(i[0])
                r1 = r2
                used = True
        if not used:
            indices.append(list_[len(list_) - 1][0])
        final_statements = [statements.loc[x].to_dict() for x in statements.index if 
                            statements.loc[x]["statement"] in indices]
        return final_statements
    else:
        return None

filtered_statements = []
for doc in tqdm(list(set(final_statements.doc))):
    statements = final_statements[final_statements["doc"] == doc]
    for sentence in list(set(statements["sentence"])):
        current_res = max_disjoint_intervals(statements, doc, sentence, condition=["ARG0_highdim", "B-V_highdim", "ARG1_highdim"])
        if current_res:
            filtered_statements.append(current_res) 

   doc  sentence  statement         ARG0_highdim ARG0_lowdim B-V_highdim  \
0    0         0          0                                         fell   
1    0         0          1  recent housing data     percent    indicate   
2    0         0          2                                                
3    0         0          3                                        start   
4    0         0          4                                      decline   

  B-V_lowdim B-ARGM-NEG_highdim B-ARGM-NEG_lowdim B-ARGM-MOD_highdim  \
0       come                                                           
1    suggest                                                           
2                                                                      
3      begin                                                           
4    decline                                                           

          ARG1_highdim ARG1_lowdim ARG2_highdim ARG2_lowdim  start  end  
0  consumer confidence     percent  

100%|██████████| 18375/18375 [42:22<00:00,  7.23it/s]


In [37]:
pickle.dump(filtered_statements, open("filtered_statements.pickle", "wb"))

In [ ]:
split_sentences = pickle.load(open("sentences.pickle", "rb"))
srl_res = pickle.load(open("srl.pickle", "rb"))
narrative_model = pickle.load(open("model.pickle", "rb"))
final_statements = pickle.load(open("final_statements.pickle", "rb"))
filtered_statements = pickle.load(open("filtered_statements.pickle", "rb"))

In [ ]:
def extend_inside(lists):
    new_list = []
    for element in lists:
        for part_of_element in element:
            new_list.append(part_of_element)
    return new_list
filtered_statements_df = pd.DataFrame(extend_inside(filtered_statements))
print(filtered_statements_df)
print(len(filtered_statements_df))

In [5]:
RELATIOn_list = ["because", "thus", "therefore", "after", "then", "if", "hence", "consequently", "resulting",
                     "since", "reason", "due"]
def filter_for_RELATIOns(doc, RELATIOns=RELATIOn_list, sentences=srl_res):
    sentence = sentences[doc]["words"]
    appearance_dict = {}
    for word in RELATIOns:
        index = [i for i, x in enumerate(sentence) if x == word]
        if len(index) > 0:
            appearance_dict[word] = index
    return appearance_dict

RELATIOns = list(map(filter_for_RELATIOns, range(len(srl_res))))

doc = -1
sentence = -1
last_end =-1
statement_list = [[] for x in range(max(split_sentences[0]) + 1)]
for iloc in range(len(filtered_statements_df)):
    statement = filtered_statements_df.iloc[iloc]
    if statement.doc > doc:
        doc = statement.doc
        sentence = statement.sentence
        last_end = -1
        current_start = statement.start
    elif statement.sentence > sentence:
        current_start = 100000
    else:
        current_start = statement.start
    current_statement = []
    if len(RELATIOns[sentence]) > 0:
        for relation in RELATIOns[sentence].keys():
            if any([x >= last_end and x < current_start for x in RELATIOns[sentence][relation]]):
                current_statement.append(relation)
    current_statement.append(statement["ARG0_highdim"])
    current_statement.append(statement["B-V_highdim"])
    current_statement.append(statement["ARG1_highdim"])
    statement_list[doc].append(current_statement)
    last_end = statement.end
    sentence = statement.sentence
    
pickle.dump(statement_list, open("RELATIOn-statements.pickle", "wb"))

In [2]:
statement_list = pickle.load(open("RELATIOn-statements.pickle", "rb"))

In [6]:
RELATIOn_list = ["because", "thus", "therefore", "after", "then", "if", "hence", "consequently", "resulting",
                     "since", "reason", "due"]
def filter_for_RELATIOns(doc, RELATIOns=RELATIOn_list, sentences=srl_res):
    sentence = sentences[doc]["words"]
    appearance_dict = {}
    for word in RELATIOns:
        index = [i for i, x in enumerate(sentence) if x == word]
        if len(index) > 0:
            appearance_dict[word] = index
    return appearance_dict

RELATIOns = list(map(filter_for_RELATIOns, range(len(srl_res))))

NameError: ignored

In [5]:
statement_list[2]

[['head start', 'provide', 'preschool program'],
 ['health human service department',
  'release',
  'study head start overall impact'],
 ['student', 'attend', 'head start'],
 ['report',
  'conclude',
  'significant impact found math skill pre write child promotion teacher report child school accomplishment ability year'],
 ['national center education statistic',
  'report',
  'nation high poverty school average graduation rate th grade student fell percent percent rate low poverty school remain stable percent'],
 ['congress', 'blame', 'belt tighten'],
 ['congress', 'tighten', 'belt']]